In [1]:
import pandas as pd
structure = pd.read_csv("course-217-structure.csv")
data = pd.read_csv("course-217-events.csv")

In [2]:
import numpy as np
import numba

In [3]:
structure.head()

,course_id,module_id,module_position,lesson_id,lesson_position,step_id,step_position,step_type,step_cost
0,217,614,2,13228,2,38842,1,text,0
1,217,614,2,13228,2,39715,6,code,1
2,217,614,2,13228,2,39716,7,code,1
3,217,614,2,13228,2,39717,8,code,1
4,217,614,2,13228,2,39721,9,free-answer,0


In [4]:
pos_df = structure.ix[:, [-4, 2, 4, 6]].sort_values("module_position")

In [5]:
from tqdm import tqdm, tqdm_notebook

In [6]:
pos_array = []

for i in range(1, 11):
    for j in range(1, 10):
        for k in range(1, 14):
            pos_array.append([i, j, k])
            
pos_dict = {}
for i, pos in enumerate(pos_array):
    pos_dict[i] = pos

In [9]:
def get_next_step_id(step_id):
    cur_pos = list(pos_df[pos_df.step_id == step_id].ix[:, [1,2,3]].values[0])
#     print(cur_pos)
    
    for key in pos_dict:
        if pos_dict[key] == cur_pos:
            i = key
            break
    
    for j in range(i + 1, 13*9*10+1):
        pos = pos_dict[j]
        if not pos_df.query("(module_position == {0}) & (lesson_position == {1}) & (step_position == {2})".format(*pos)).empty:
            return pos_df.query("(module_position == {0}) & (lesson_position == {1}) & (step_position == {2})".format(*pos)).step_id

In [ ]:
def f(x):
    res = []
    try:
        for step_id in np.unique(x.step_id):
            first = second = third = False

            next_step_id = get_next_step_id(step_id).values[0]
            if next_step_id:
                for cur_step_id in x.step_id:
                    if first and second and third:
                        res.append(step_id)

                    if cur_step_id == step_id and not first:
                        first = True
                    if cur_step_id == next_step_id and not second:
                        second = True
                    if cur_step_id == step_id and not third:
                        third = True
    except:
        pass
    return res

tqdm.pandas()
returnable_users = data.sort_values("time").groupby("user_id").progress_apply(f)




  0%|          | 0/7663 [00:00<?, ?it/s]


  0%|          | 1/7663 [00:00<17:58,  7.10it/s]


  0%|          | 2/7663 [00:02<1:46:49,  1.20it/s]


  0%|          | 3/7663 [00:04<2:33:55,  1.21s/it]


  0%|          | 4/7663 [00:06<2:46:27,  1.30s/it]


  0%|          | 5/7663 [00:07<2:49:43,  1.33s/it]


  0%|          | 6/7663 [00:07<2:07:21,  1.00it/s]


  0%|          | 7/7663 [00:08<2:12:25,  1.04s/it]


  0%|          | 8/7663 [00:11<3:02:26,  1.43s/it]


  0%|          | 9/7663 [00:13<3:21:24,  1.58s/it]


  0%|          | 10/7663 [00:14<3:09:40,  1.49s/it]


  0%|          | 11/7663 [00:15<2:33:43,  1.21s/it]


  0%|          | 12/7663 [00:16<2:47:17,  1.31s/it]


  0%|          | 13/7663 [00:19<3:29:51,  1.65s/it]


  0%|          | 14/7663 [00:19<2:53:39,  1.36s/it]


  0%|          | 15/7663 [00:20<2:22:10,  1.12s/it]


  0%|          | 16/7663 [00:21<2:39:52,  1.25s/it]


  0%|          | 17/7663 [00:21<1:56:51,  1.09it/s]


  0%|          | 18/7663 [00:22<1:57:30,  1.08i

In [18]:
returnable_users = [np.unique(x) for x in returnable_users]

In [20]:
from collections import Counter
c = Counter()

for ar in returnable_users:
    c.update(ar)


In [32]:
coeffs = [(key, c[key] / len(data[data.step_id == key].groupby("user_id"))) for key in c]

In [33]:
res = sorted(coeffs, reverse=True, key=lambda x: x[1])[:10]
res
",".join([str(x[0]) for x in res])

[(47438, 0.9887459807073955),
 (44391, 0.9882943143812709),
 (38528, 0.9881586737714624),
 (39236, 0.9846857597454256),
 (48838, 0.9827089337175793),
 (48842, 0.9826589595375722),
 (47290, 0.9823204419889503),
 (42398, 0.9822161422708618),
 (38525, 0.9813411078717201),
 (38524, 0.980846581114729)]

'47438,44391,38528,39236,48838,48842,47290,42398,38525,38524'

In [31]:
len(data[data.step_id == 44120].groupby("user_id"))

1125